<a href="https://colab.research.google.com/github/impanaj07/deep_learning/blob/main/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import os

# Define parameters
img_size = (32, 32)  # CIFAR-10 image size
batch_size = 32
num_classes = 10  # CIFAR-10 has 10 classes
epochs =  5 # Adjust as needed

# Load CIFAR-10 dataset
(x_train, y_train_raw), (x_val, y_val_raw) = keras.datasets.cifar10.load_data()

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train_raw, num_classes=num_classes)
y_val = tf.keras.utils.to_categorical(y_val_raw, num_classes=num_classes)

# Data Augmentation and Preprocessing
datagen_train = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    # validation_split is not used here as we have a separate validation set
)

datagen_val = ImageDataGenerator(rescale=1.0/255)

train_generator = datagen_train.flow(
    x_train, y_train,  # Use one-hot encoded labels here
    batch_size=batch_size
)

val_generator = datagen_val.flow(
    x_val, y_val,    # Use one-hot encoded labels here
    batch_size=batch_size
)

# Load Pretrained Model (ResNet50 does not have pre-trained weights for CIFAR-10)
# We will use weights='imagenet' and fine-tune
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
base_model.trainable = False  # Freeze the base model

# Add Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
out = Dense(num_classes, activation='softmax')(x)

# Compile Model
model = Model(inputs=base_model.input, outputs=out)
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    steps_per_epoch=len(x_train) // batch_size,
    validation_steps=len(x_val) // batch_size
)

# Fine-tune the model by unfreezing some layers
base_model.trainable = True
for layer in base_model.layers[:100]:  # Keep some layers frozen
    layer.trainable = False

# Compile again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])

# Train again for fine-tuning
model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs // 2,
    steps_per_epoch=len(x_train) // batch_size,
    validation_steps=len(x_val) // batch_size
)

# Save Model
model.save("fine_tuned_resnet50_cifar10.h5")

Epoch 1/5
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 58s 30ms/step - accuracy: 0.1058 - loss: 2.3946 - val_accuracy: 0.0991 - val_loss: 2.3026
Epoch 2/5
   1/1562 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - accuracy: 0.0625 - loss: 2.3025

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


1562/1562 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0625 - loss: 2.3025 - val_accuracy: 0.0981 - val_loss: 2.3026
Epoch 3/5
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 66s 25ms/step - accuracy: 0.1020 - loss: 2.3028 - val_accuracy: 0.1004 - val_loss: 2.3027
Epoch 4/5
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.0625 - loss: 2.3079 - val_accuracy: 0.1004 - val_loss: 2.3027
Epoch 5/5
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 82s 27ms/step - accuracy: 0.0983 - loss: 2.3027 - val_accuracy: 0.1000 - val_loss: 2.3026
Epoch 1/2
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 90s 39ms/step - accuracy: 0.1765 - loss: 2.3790 - val_accuracy: 0.1441 - val_loss: 2.3342
Epoch 2/2
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.3125 - loss: 2.0645 - val_accuracy: 0.1437 - val_loss: 2.3282
